# kafkaReceiveDataPy
This notebook receives data from Kafka on the topic 'test', and stores it in the 'time_test' table of Cassandra (created by cassandra_init.script in startup_script.sh).

```
CREATE KEYSPACE test_time WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};

CREATE TABLE test_time.sent_received(
 time_sent TEXT,
 time_received TEXT,
PRIMARY KEY (time_sent)
);
```

A message that gives the current time is received every second. 

## Add dependencies

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'
import time

## Load modules and start SparkContext
Note that SparkContext must be started to effectively load the package dependencies. Two cores are used, since one is needed for running the Kafka receiver.

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "localhost")
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

## SaveToCassandra function
Takes a list of tuple (rows) and save to Cassandra 

In [3]:
def saveToCassandra(rows):
    if not rows.isEmpty(): 
        sqlContext.createDataFrame(rows).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table="region", keyspace="palissy")\
        .save()

## Create streaming task
* Receive data from Kafka 'test' topic every five seconds
* Get stream content, and add receiving time to each message
* Save each RDD in the DStream to Cassandra. Also print on screen

In [4]:
ssc = StreamingContext(sc, 5)
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'test': 1})
data = kvs.map(lambda x: x[1].split(','))
rows= data.map(lambda x:Row(id=int(x[0]),code=x[1],name=x[2],slug=x[3]))
rows.foreachRDD(saveToCassandra)
rows.pprint()

## Start streaming

In [5]:
ssc.start()

## Stop streaming

In [6]:
#ssc.stop(stopSparkContext=False,stopGraceFully=True)

-------------------------------------------
Time: 2019-03-28 10:05:40
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:05:45
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:05:50
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:05:55
-------------------------------------------
Row(code=u'01', id=1, name=u'Guadeloupe', slug=u'guadeloupe\n')
Row(code=u'02', id=2, name=u'Martinique', slug=u'martinique\n')
Row(code=u'03', id=3, name=u'Guyane', slug=u'guyane\n')
Row(code=u'04', id=4, name=u'"La R\xe9union"', slug=u'"la reunion"\n')
Row(code=u'06', id=5, name=u'Mayotte', slug=u'mayotte\n')
Row(code=u'11', id=6, name=u'\xcele-de-France', slug=u'"ile de france"\n')
Row(code=u'24', id=7, name=u'"Centre-Val de Loire"', slug=u'"centre val de loire"\n')
Row(code=u'27', id=8, name=u'Bourgogne-Franche-Comt\xe9', slug=u'"b

-------------------------------------------
Time: 2019-03-28 10:11:10
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:15
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:20
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:25
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:30
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:35
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:40
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:45
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:11:50
----------

-------------------------------------------
Time: 2019-03-28 10:17:10
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:15
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:20
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:25
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:30
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:35
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:40
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:45
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:17:50
----------

-------------------------------------------
Time: 2019-03-28 10:23:10
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:15
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:20
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:25
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:30
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:35
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:40
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:45
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:23:50
----------

-------------------------------------------
Time: 2019-03-28 10:29:10
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:15
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:20
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:25
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:30
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:35
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:40
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:45
-------------------------------------------

-------------------------------------------
Time: 2019-03-28 10:29:50
----------

## Get Cassandra table content

## Get Cassandra table content using SQL